In [1]:
__author__ = 'Monika Soraisam'
__email__ = 'monika.soraisam@noirlab.edu'

# This notebook is meant for generating trimmed files (NIRI imaging) for unit testing for Dan (https://noirlab.atlassian.net/jira/software/projects/GOATS/boards/57?selectedIssue=GOATS-372)

In [2]:
import os
import sys
from pathlib import Path
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime
import types

In [3]:
# Now import the DRAGONS libraries 
import astrodata
import gemini_instruments
from recipe_system import cal_service
from gempy.adlibrary import dataselect
from gempy.utils import logutils

from recipe_system.reduction.coreReduce import Reduce
from gempy.scripts import showpars
from gempy.utils.showrecipes import showrecipes
from gempy.utils.showrecipes import showprims

In [4]:
## prep the reduction folder
def prep_reduction_folder(data_root, obsid):
    reduction_path = Path(f"{data_root}/{obsid}/reduction")

    if not reduction_path.exists():
        os.mkdir(reduction_path.as_posix())
        print (f"directory for dragons reduction created")
    
    ## change the cwd to the reduction folder
    os.chdir(reduction_path.as_posix())
    print(f"Current working directory is: {os.getcwd()}")

    ## write the configuration file 
    mydb = "dragons_for_goats.db" 
    mydb_path = reduction_path.as_posix() + '/' + mydb
    print (mydb_path)
    
    dragons_rc = reduction_path.as_posix() + '/dragonsrc'
    print (dragons_rc)
    
    with open(dragons_rc, "w") as f:
        f.write("[calibs]\ndatabases = {0} get store".format(mydb_path))

    return dragons_rc, mydb_path


In [5]:
data_path = "/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM"
obsid = 'GN-2016B-FT-16-9'  # NIRI imaging example 
dragons_rc, mydb_path = prep_reduction_folder(data_path, obsid)

Current working directory is: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/reduction
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/reduction/dragons_for_goats.db
/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/reduction/dragonsrc


In [6]:
def generate_filelists(location, obsid):
    """
    Parameters
    ----------
    location: str
        Root folder where the Gemini data for a given target is located
    obsid: str
        Gemini observation ID 
    """

    
    all_files = [str(pp) for pp in list(Path(location+"/"+obsid).glob('*.fits'))]
    all_files.sort()
    print (f'The total number of files for observation ID {obsid} is {len(all_files)}')

    obs_types = ['OBJECT','BIAS','DARK','FLAT','ARC','PINHOLE','RONCHI','CAL','FRINGE','MOS MASK', 'BPM'] #fetched from Obs Type search field on GOA, which is relevant for DRAGONS

    all_meta = {}

    # Note that even if a few keywords are extracted below, thiis is just meant for demonstration purposes. 
    # If onyl these keywords are extracted in production, it will not align with  
    # implementation of item (2), allowing users to group/select preferred header keywords
    for K in obs_types:
        all_meta[K] = {'file':[],
                        'obs_class':[],
                        'group_id':[],
                        'exp':[],
                        'object':[],
                        'wave':[],
                        'waveband':[],
                        'date':[],
                        'roi':[],
                        }

    object_files = []
    for i,F in enumerate(all_files):
        ad = astrodata.open(F)

        if "BPM" in ad.tags or "UNPREPARED" in ad.tags: ## want only raw, i.e., "unprepared" files, but BPM is an exception, which is processed/prepared 
            K = ad.observation_type() ## astrodata header also has the observation type, which should match what's in the archive drop-down menu
        
        elif "PREPARED" in ad.tags or "PROCESSED" in ad.tags: ## skip all "prepared"/"processed" files
            continue
            
        all_meta[K]['file'].append(F)
        all_meta[K]['obs_class'].append(ad.observation_class())
        
        # group_id seems to be not implemented for GNIRS spectroscopy yet
        if "GNIRS" in ad.instrument():
            all_meta[K]['group_id'].append(None)
        else:
            all_meta[K]['group_id'].append(ad.group_id())
        all_meta[K]['exp'].append(ad.exposure_time())
        all_meta[K]['object'].append(ad.object())
        all_meta[K]['wave'].append(ad.central_wavelength())
        all_meta[K]['waveband'].append(ad.wavelength_band())
        all_meta[K]['date'].append(ad.ut_date())
        all_meta[K]['roi'].append(ad.detector_roi_setting()) 
        #print (F.split('/')[-1], ad.object(), ad.tags)
    
    return all_meta


In [7]:
all_meta = generate_filelists(data_path, obsid)

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}") 

The total number of files for observation ID GN-2016B-FT-16-9 is 109
There are 42 files for observation type OBJECT
There are 10 files for observation type DARK
There are 56 files for observation type FLAT
There are 1 files for observation type BPM


In [8]:
DF_object = pd.DataFrame(all_meta['OBJECT'])
DF_dark = pd.DataFrame(all_meta['DARK'])
DF_flat = pd.DataFrame(all_meta['FLAT'])
DF_bpm = pd.DataFrame(all_meta['BPM'])

In [9]:
print (f"There are {len(np.unique(DF_object['object'].values))} unique objects in this observation set, namely {np.unique(DF_object['object'].values)}")

There are 3 unique objects in this observation set, namely ['FS 3' 'FS-103' 'iPTF-16fnl']


## We will use astrodata to "write" the trimmed arrary files for each observation type but retaining the same header info. 

In [10]:
## for dark

ad = astrodata.open(DF_dark['file'].values[0])
ad.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/N20161012S0380.fits
Tags: AT_ZENITH AZEL_TARGET CAL DARK GEMINI NIRI NON_SIDEREAL NORTH RAW
    UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (1024, 1024)   float32


In [12]:
# Let's reduce the data array size by a factor 8
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]

In [13]:
# write it to a test file
ad.write('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_dark.fits', overwrite=True)

In [14]:
# check the header is all good
AD = astrodata.open('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_dark.fits')
AD.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_dark.fits
Tags: AT_ZENITH AZEL_TARGET CAL DARK GEMINI NIRI NON_SIDEREAL NORTH RAW
    UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (128, 128)     float32


In [15]:
## for flat
ad = astrodata.open(DF_flat['file'].values[0])
ad.info()


Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/N20161012S0344.fits
Tags: AT_ZENITH AZEL_TARGET CAL FLAT GCALFLAT GCAL_IR_OFF GEMINI IMAGE LAMPOFF
    NIRI NON_SIDEREAL NORTH RAW UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (1024, 1024)   float32


In [16]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_flat.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_flat.fits')
AD.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_flat.fits
Tags: AT_ZENITH AZEL_TARGET CAL FLAT GCALFLAT GCAL_IR_OFF GEMINI IMAGE LAMPOFF
    NIRI NON_SIDEREAL NORTH RAW UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (128, 128)     float32


In [18]:
## for bpm
ad = astrodata.open(DF_bpm['file'].values[0])
ad.info()


Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/bpm_20010317_niri_niri_11_full_1amp.fits
Tags: BPM CAL GEMINI NIRI NORTH PREPARED PROCESSED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (1024, 1024)   int32


In [19]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_bpm.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_bpm.fits')
AD.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_bpm.fits
Tags: BPM CAL GEMINI NIRI NORTH PREPARED PROCESSED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (128, 128)     int32


In [20]:
## for object FS 3
temp = DF_object[DF_object['object']=='FS 3']
ad = astrodata.open(temp['file'].values[0])
print (f"Object is {ad.object()}")
ad.info()

Object is FS 3
Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/N20161013S0230.fits
Tags: GEMINI IMAGE NIRI NORTH RAW SIDEREAL UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (1024, 1024)   float32


In [21]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_FS_3.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_FS_3.fits')
AD.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_FS_3.fits
Tags: GEMINI IMAGE NIRI NORTH RAW SIDEREAL UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (128, 128)     float32


In [22]:
## for object FS-103
temp = DF_object[DF_object['object']=='FS-103']
ad = astrodata.open(temp['file'].values[0])
print (f"Object is {ad.object()}")
ad.info()

Object is FS-103
Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/N20161012S0226.fits
Tags: GEMINI IMAGE NIRI NORTH RAW SIDEREAL UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (1024, 1024)   float32


In [23]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_FS_103.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_FS_103.fits')
AD.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_FS_103.fits
Tags: GEMINI IMAGE NIRI NORTH RAW SIDEREAL UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (128, 128)     float32


In [24]:
## for object iPTF-16fnl
temp = DF_object[DF_object['object']=='iPTF-16fnl']
ad = astrodata.open(temp['file'].values[0])
print (f"Object is {ad.object()}")
ad.info()

Object is iPTF-16fnl
Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18abtvjfy/GEM/GN-2016B-FT-16-9/N20161012S0236.fits
Tags: GEMINI IMAGE NIRI NORTH RAW SIDEREAL UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (1024, 1024)   float32


In [25]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_iPTF-16fnl.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('/Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_iPTF-16fnl.fits')
AD.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/goats/scripts/test_files/niri_imaging_object_iPTF-16fnl.fits
Tags: GEMINI IMAGE NIRI NORTH RAW SIDEREAL UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (128, 128)     float32
